# How to Generate Music using a LSTM Neural Network in PyTorch

[How to Generate Music using a LSTM Neural Network in Keras](https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5)を訳しながらPyTorchで実装を目指します。

データセットは[The Largest MIDI Collection on the Internet](https://www.reddit.com/r/WeAreTheMusicMakers/comments/3ajwe4/the_largest_midi_collection_on_the_internet/)
- 公開されているMIDIデータを収集した大規模なデータセット(※もちろん有料なコンテンツは含まれない)。
- ポップ、クラシック、ゲーム音楽など多彩なジャンルで構成されており、総ファイル数13万・約100時間分のデータとなっている。
- Tronto大学の[Song From PI](http://www.cs.toronto.edu/songfrompi/)で使用されたデータセット


## Introduction

近年、ニューラルネットワークを使ってテキストを生成する方法についてのチュートリアルは数多くありますが、音楽を作成する方法についてのチュートリアルは不足しています。
この記事では、Kerasライブラリを使ってPythonでリカレントニューラルネットワークを使って音楽を作成する方法を解説します。
せっかちな方のために、チュートリアルの最後にGithubリポジトリへのリンクがあります。

## 背景
実装の詳細に入る前に、明確にしておかなければならない用語があります。

### LSTM
このチュートリアルでは、Long Short-Term Memory (LSTM)ネットワークを使用します。
LSTMはリカレント・ニューラル・ネットワークの一種で、勾配降下を介して効率的に学習することができます。
ゲーティング・メカニズムを用いて、LSTMは長期パターンを認識し、符号化することができます。
LSTMは、音楽やテキスト生成のように、ネットワークが長期間にわたって情報を記憶しなければならないような問題を解決するのに非常に有用です。

### Music21
Music21は、コンピュータ支援音楽学に使われるPythonのツールキットです。音楽理論の基礎を教えたり、譜例を生成したり、音楽を勉強したりすることができます。
このツールキットは、MIDIファイルの楽譜を取得するためのシンプルなインターフェースを提供します。
さらに、ノートとコードオブジェクトを作成して、自分のMIDIファイルを簡単に作成することができます。
このチュートリアルでは、Music21を使ってデータセットの内容を抽出し、ニューラルネットワークの出力を取り、楽譜に変換します。

### トレーニング
このセクションでは、モデルのデータをどのように収集したか、LSTMモデルで使用できるようにデータをどのように準備したか、モデルのアーキテクチャについて説明します。
データ
私たちのGithubリポジトリでは、主にファイナルファンタジーのサウンドトラックからの音楽で構成されたピアノ音楽を使用しました。私たちがファイナルファンタジーの音楽を選んだのは、作品の大部分が持っている非常にはっきりとした美しいメロディーと、存在する作品の量の多さのためです。しかし、単一の楽器で構成されたMIDIファイルのセットであれば、どのようなものでも構いません。
ニューラルネットワークを実装するための最初のステップは、作業するデータを調べることです。

下の図は、Music21を使って読み込んだMIDIファイルの抜粋です。

```
<music21.note.Note B> 72.0
<music21.chord.Chord E3 A3> 72.0
<music21.note.Note A> 72.5
<music21.chord.Chord E3 A3> 72.5
<music21.note.Note E> 73.0
<music21.chord.Chord E3 A3> 73.0
<music21.chord.Chord E3 A3> 73.5
<music21.note.Note E-> 74.0
<music21.chord.Chord F3 A3> 74.0
<music21.chord.Chord F3 A3> 74.5
<music21.chord.Chord F3 A3> 75.0
<music21.chord.Chord F3 A3> 75.5
<music21.chord.Chord E3 A3> 76.0
<music21.chord.Chord E3 A3> 76.5
<music21.chord.Chord E3 A3> 77.0
<music21.chord.Chord E3 A3> 77.5
<music21.chord.Chord F3 A3> 78.0
<music21.chord.Chord F3 A3> 78.5
<music21.chord.Chord F3 A3> 79.0
```

この抜粋とデータセットのほとんどからわかるように、ミディファイルのノート間の最も一般的な間隔は 0.5 です。したがって、可能な出力のリストの中の様々なオフセットを無視することで、データとモデルを単純化することができます。
これは、ネットワークによって生成された音楽のメロディーにあまり深刻な影響を与えません。
そこで、このチュートリアルではオフセットを無視して、可能な出力のリストを352にしておきます。

#### データの準備
データを調べて、LSTMネットワークの入力と出力として使用したい機能が音符と和音であることがわかったので、ネットワーク用のデータを準備します。

まず、以下のコードスニペットにあるように、データを配列に読み込みます。

```
from music21 import converter, instrument, note, chord
notes = []
for file in glob.glob("midi_songs/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
```

まず、converter.parse(file)関数を使って、各ファイルをMusic21のストリームオブジェクトにロードします。
このストリームオブジェクトを使って、ファイル内のすべての音符と和音のリストを取得します。
音の最も重要な部分は音程の文字列表記を使って再現できるので、すべての音符オブジェクトの音程を文字列表記を使って追加します。
そして、コードの中のすべてのノートのIDを、それぞれのノートをドットで区切って1つの文字列にエンコードすることで、すべてのコードを追加します。
これらのエンコーディングにより、ネットワークで生成された出力を正しい音と和音に簡単にデコードすることができます。

これで、すべての音符と和音をシーケンシャルリストにまとめたので、ネットワークの入力となるシーケン スを作成できます。

![midimusic.jpg](./images/midimusic.jpg)

<center>図1: カテゴリから数値データに変換するとき、データは、カテゴリが明確な値のセットの中でどこに位置しているかを表す整数のインデックスに変換されます。例えば、リンゴは最初の明確な値なので0にマップされ、オレンジは2番目なので1にマップされ、パイナップルは3番目なので2にマップされます。</center>

まず、文字列ベースのカテゴリデータから整数ベースの数値データへのマッピング関数を作成します。
これは、ニューラルネットワークが文字列ベースのカテゴリデータよりも整数ベースの数値データの方が性能が良いためです。
カテゴリデータから数値データへの変換の例を図1に示します。


次に、ネットワークの入力シーケンスとそれぞれの出力を作成します。
各入力シーケンスの出力は、入力シーケンスの音符のリストの中で、入力シーケンスの音符のシーケンスの後に来る最初の音符または和音になります。

```python
sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)
```

このコード例では、各シーケンスの長さを100音/和音としています。
これは、シーケンスの次の音を予測するために、ネットワークが予測を行うのに役立つ前の100音を持っていることを意味します。
シーケンスの長さの違いがネットワークによって生成された音楽に与える影響を確認するために、異なるシーケンスの長さを使用してネットワークをトレーニングすることを強くお勧めします。
ネットワーク用のデータを準備する最後のステップは、入力を正規化し、出力を[ワンホットエンコード](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/)することです。

## Model

最後に、モデルアーキテクチャの設計に入ります。
このモデルでは、4つの異なるタイプのレイヤーを使用しています。

LSTM層は、シーケンスを入力として受け取り、シーケンス(return_sequences=True)または行列を返すリカレント・ニューラル・ネット層です。

ドロップアウト層は正則化技術で、オーバーフィットを防ぐために、訓練中の各更新時に入力単位の端数を0に設定することで構成されています。
この割合は，その層で使用されるパラメータによって決定されます。

密層(Dense layers)または完全接続層(fully connected layers)は、各入力ノードが各出力ノードに接続されている完全接続型のニューラルネットワーク層です。

活性化層は、我々のニューラルネットワークがノードの出力を計算するために使用する活性化関数を決定します。

```python
model = Sequential()
    model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
```
これで、使用するさまざまなレイヤについての情報が得られたので、それらをネットワークモデルに追加します。

各LSTM, Dense, Activation層の最初のパラメータは、その層が持つべきノードの数です。ドロップアウト層では、最初のパラメータは学習中にドロップアウトされる入力ユニットの割合です。

最初のレイヤーでは、input_shapeと呼ばれるユニークなパラメータを与えなければなりません。このパラメータの目的は、学習するデータの形状をネットワークに知らせることです。

最後の層には、システムが持つ異なる出力の数と同じ数のノードを常に含まなければなりません。これにより、ネットワークの出力がクラスに直接対応することが保証されます。

このチュートリアルでは、3つのLSTM層、3つのドロップアウト層、2つの密層、1つの活性化層からなる単純なネットワークを使用します。予測の質を向上させることができるかどうかを確認するために、ネットワークの構造で遊んでみることをお勧めします。

学習の各反復の損失を計算するために、我々の出力のそれぞれが単一のクラスに属するだけであり、作業するクラスが2つ以上あるので、カテゴリークロスエントロピーを使用します。そして、ネットワークを最適化するためにRMSpropオプティマイザを使用します。

```python
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)
```
ネットワークのアーキテクチャを決定したら、トレーニングを開始します。
Kerasのmodel.fit()関数を使ってネットワークを学習します。最初のパラメータは先ほど準備した入力シーケンスのリストで、2番目のパラメータはそれぞれの出力のリストです。
このチュートリアルでは、ネットワークを200エポック（反復）、ネットワークを伝搬する各バッチには64個のサンプルが含まれています。

努力の成果を失うことなく、いつでも学習を中止できるようにするために、モデル・チェックポイントを使用します。モデル・チェックポイントは、エポックごとにネットワーク・ノードの重みをファイルに保存する方法を提供してくれます。
これにより、重みを失うことを心配することなく、損失値に満足したらニューラル・ネットワークの実行を停止することができます。
そうでなければ、重みをファイルに保存する機会を得る前に、ネットワークが200エポックすべてを通過し終わるまで待たなければなりません。

## 音楽の生成
ネットワークのトレーニングが終わったので、今まで何時間もかけてトレーニングしてきたネットワークを楽しんでみましょう。
ニューラルネットワークを使って音楽を生成できるようにするには、以前と同じ状態にする必要があります。
簡単にするために、トレーニングセクションのコードを再利用してデータを準備し、前と同じ方法でネットワークモデルをセットアップします。ただし、ネットワークをトレーニングする代わりに、トレーニングセクションで保存した重みをモデルにロードします。

```python
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
model.load_weights('weights.hdf5')
```
ここで、訓練されたモデルを使用して、ノートの生成を開始できます。

自由に使えるノートシーケンスの完全なリストがあるので、リスト内のランダムなインデックスを開始点として選択します。
しかし、開始点を制御したい場合は、ランダム関数をコマンドライン引数に置き換えてください。

ここでは、ネットワークの出力をデコードするためのマッピング関数も作成する必要があります。
この関数は数値データからカテゴリデータ（整数からノートまで）へのマッピングを行います。

```python
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
````
私たちは、ネットワークを使って500音を生成することにしました。
生成したい音符ごとに、ネットワークにシーケンスを送信しなければなりません。
最初に提出するシーケンスは、開始インデックスにある音符のシーケンスです。
それ以降のシーケンスを入力として使用するたびに、図2に示すように、シーケンスの最初の音符を削除し、シーケンスの最後に前の反復の出力を挿入します。

![sequence](./images/sequence.jpg)
<center>図2: 最初の入力シーケンスはABCDEです。次の反復では、シーケンスからAを削除してFを追加します。そして、この処理を繰り返します。</center>

ネットワークからの出力から最も可能性の高い予測を決定するために、最も高い値のインデックスを抽出します。
出力配列のインデックスXの値は、Xが次の音符である確率に対応しています。
これを説明するのに役立つのが図3です。

![mapping](./images/mapping.jpg)
<center>図3: ここでは、ネットワークからの出力予測とクラスの間のマッピングを示しています。最も高い確率で次の値がDであることがわかるので、最も確率の高いクラスとしてDを選択します。</center>

次に、ネットワークからのすべての出力を 1 つの配列に集めます。

これで、音符と和音のエンコードされた表現がすべて配列になったので、それらをデコードして、音符と和音のオブジェクトの配列を作成することができます。

まず、デコードしている出力がノートなのかコードなのかを判断しなければなりません。

パターンがコードの場合、文字列を音符の配列に分割しなければなりません。そして、それぞれのノートの文字列表現をループして、それぞれのノートのノートオブジェクトを作成します。そして、これらのノートのそれぞれを含むコードオブジェクトを作成します。

パターンがノートの場合は、パターンに含まれる音程の文字列表現を使用してノートオブジェクトを作成します。

各反復の最後に、オフセットを0.5増加させ（前節で決めたように）、作成したNote/Chordオブジェクトをリストに追加します。

```python
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5
```
ネットワークで生成された音符と和音のリストができたので、このリストをパラメータにして Music21 Stream オブジェクトを作成します。
最後に、ネットワークで生成された音楽を含むMIDIファイルを作成するために、Music21ツールキットのwrite関数を使って、ストリームをファイルに書き出します。

```python
midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='test_output.mid')
```
## 結果
さて、その結果に驚嘆する時が来ました。
図4は、LSTMネットワークを使用して生成された音楽の楽譜表現を含んでいます。
ぱっと見ただけで、それにはいくつかの構造があることがわかります。
これは、2ページ目の3行目から最後の行で特に明らかです。

音楽の知識があり、楽譜を読むことができる人は、楽譜に奇妙な音符が散らばっていることがわかります。
これはニューラルネットワークが完璧なメロディーを作ることができない結果です。
現在の実装では、常にいくつかの偽の音符があり、より良い結果を得るためには、より大きなネットワークが必要になります。

![example_of_sheet_music](./images/example_of_sheet_music.png)
<center>図4：LSTMネットワークで生成された楽譜の一例</center>

この比較的浅いネットワークの結果は、Embed 1の音楽の例を見ても分かるように、非常に印象的です。 
興味のある方は、図4の楽譜は、NeuralNet Music 5の楽譜を表しています。

## 今後の仕事
簡単なLSTMネットワークと352クラスを使って、目覚ましい成果と美しいメロディを実現しています。
しかし、改善できる部分もあります。

まず、現在の実装では、音符の長さを変えたり、音符間のオフセットを変えたりすることはできません。
これを実現するためには、異なる持続時間ごとにクラスを追加し、音符間の休符期間を表す休符クラスを追加することができます。

より多くのクラスを追加して満足のいく結果を得るためには、LSTMネットワークの深さを増やす必要があり、そのためにはかなり強力なコンピュータが必要になります。
私が自宅で使っているノートパソコンでは、現在のようにネットワークを学習するのに約20時間かかりました。

第二に、ピースに始まりと終わりを追加します。現在のネットワークのように、ピースの間には区別がありません。
これにより、ネットワークは、現在のように生成されたピースを突然終了させるのではなく、最初から最後までピースを生成することができるようになります。

第三に、不明なノートを処理する方法を追加します。
現在、ネットワークは知らない音に遭遇するとフェイル状態になります。
この問題を解決する方法としては、未知のノートに最も近いノートやコードを見つけることが考えられます。

最後に、データセットに楽器を追加します。
現在のところ、ネットワークがサポートしているのは単一の楽器のみです。
オーケストラ全体をサポートするためにネットワークを拡張できたら面白いでしょう。

## 結論
このチュートリアルでは、音楽を生成するLSTMニューラルネットワークを作成する方法を示しました。
結果は完璧ではないかもしれませんが、それにもかかわらず、かなり印象的なものであり、ニューラルネットワークが音楽を作成し、より複雑な音楽を作成するのに役立つ可能性があることを示しています。

[チュートリアルのGithubリポジトリをチェックしてみてください。](https://github.com/Skuldur/Classical-Piano-Composer)